In [ ]:
import sqlalchemy as db

In [ ]:
engine = db.create_engine("sqlite:///european_database.sqlite")
conn = engine.connect()

In [ ]:
#extracting the metadata
metadata = db.MetaData()

#Table object
division = db.Table('divisions', metadata, autoload_with=engine)
match = db.Table('matchs', metadata, autoload_with=engine)

In [ ]:
print(repr(metadata.tables['divisions']))
print(repr(metadata.tables['matchs']))

In [ ]:
print(division.columns.keys())

In [ ]:
#SELECT * FROM divisions
query = division.select() 
print(query)

In [ ]:
# or
print(db.select(division))

### SQL query result

fetchone(): it will extract a single row at a time.

fetchmany(n): it will extract the n number of rows at a time.

fetchall(): it will extract all of the rows. 

In [ ]:
#executing the query using connection object
exe = conn.execute(query)

#extracting top 5 results
result = exe.fetchmany(5) 
print(result)

In [ ]:
match = db.Table('matchs', metadata, autoload_with=engine)
query = match.select()

print(query)

In [ ]:
print(match.columns.keys())

In [ ]:
exe = conn.execute(query)
result = exe.fetchmany(5) 
print(result)

In [ ]:
print(division.columns.keys())
print(match.columns.keys())

Running complex query

In [ ]:
import pandas as pd

query = (db.select(division, match).
         join(match,division.columns.division == match.columns.Div).
         where(db.and_(division.columns.division == "E1", match.columns.season == 2009 )).
         order_by(match.columns.HomeTeam))

output = conn.execute(query)
results = output.fetchall()

data = pd.DataFrame(results)
data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

f, ax = plt.subplots(figsize=(15, 6))
plt.xticks(rotation=90)
sns.set_color_codes("pastel")
sns.barplot(x="HomeTeam", y="FTHG", data=data,
            label="Home Team Goals", color="b")

sns.barplot(x="HomeTeam", y="FTAG", data=data,
            label="Away Team Goals", color="r")
ax.legend(ncol=2, loc="upper left", frameon=True)
ax.set(ylabel="", xlabel="")
sns.despine(left=True, bottom=True)

Baseball

In [ ]:
engine = db.create_engine("sqlite:///baseball.sqlite")

In [ ]:
import pandas as pd

df = pd.read_csv('dataset/AllstarFull.csv')
df.to_sql(con=engine, name="AllstarFull", if_exists='replace', index=False)

In [ ]:
conn = engine.connect()
metadata = db.MetaData()

In [ ]:
baseball = db.Table('allstarfull', metadata, autoload_with=engine)

In [ ]:
baseball.columns.keys()

In [ ]:
query = db.select(baseball)
result = conn.execute(query)
result.fetchmany(5)

In [ ]:
df = pd.DataFrame(result.fetchmany(5))
df

In [ ]:
query = db.select(baseball).where(
    db.and_((baseball.columns.teamID=='NYA'), baseball.columns.yearID>2000)
    )

result = conn.execute(query)

df = pd.DataFrame(result.fetchmany(5))
df

In [ ]:
query = db.select(
    baseball.c.teamID, 
    db.func.count(baseball.c.playerID).label('NoOfPlayers'))\
        .group_by(baseball.c.teamID)

result = conn.execute(query)
df = pd.DataFrame(result.fetchmany(5))
df

In [ ]:
query = db.select(
    baseball.c.teamID, 
    db.func.count(baseball.c.playerID).label('NoOfPlayers'))\
        .group_by(baseball.c.teamID)\
        .order_by(db.desc('NoOfPlayers'))

result = conn.execute(query)
df = pd.DataFrame(result.fetchmany(5))
df

SQL Alchemy ORM Quick Start

In [ ]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [ ]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///user.db", echo=True)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

In [ ]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)

In [ ]:
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == "sandy")
    .where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()

In [ ]:
sandy_address

In [ ]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()

In [ ]:
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))

In [ ]:
sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"
session.commit()

In [ ]:
sandy = session.get(User, 2)

In [ ]:
sandy.addresses.remove(sandy_address)

In [ ]:
session.flush()

In [ ]:
session.delete(patrick)

In [ ]:
session.commit()

https://docs.sqlalchemy.org/en/20/orm/quickstart.html
https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial
https://docs.sqlalchemy.org/en/20/dialects/mssql.html#module-sqlalchemy.dialects.mssql.pyodbc